In [82]:
using LinearAlgebra



#true position of target 
#note zero
#acceleration


In [83]:
# Example Here
A = [1 0 1 0; 0 1 0 1; 0 0 1 0; 0 0 0 1]
sigma = 0.09774


truex = [1.4, 1.2, 1.0, 0.8]
truey = [1.65, 1.05, 0.45, -0.15]
truetheta1 = pi * (truey .< 0) .+ atan.(truex./truey);
truetheta2 = pi * (truey .> 1) .+ atan.(truex./(1 .- truey))
theta1 = (truetheta1 .+ randn(4)*sigma)
theta2 = (truetheta2 .+ randn(4)*sigma)
x = (sin.(theta1) .* sin.(theta2) ) ./ (sin.(theta1 .+ theta2))
y = (cos.(theta1) .* sin.(theta2) ) ./ (sin.(theta1 .+ theta2));


In [84]:
#here are the observations x and y

vx = similar(x)
vx[1] = 0
for i in 2:length(x)
     vx[i] = x[i] - x[i-1]
end

vy = similar(y)
vy[1] = 0
for i in 2:length(y)
     vy[i] = y[i] - y[i-1]
end




In [85]:
# Calculate "Process Covariance Matrix 

# A*Pk0*A'

dfdtheta1 = (1 ./ sin.(theta1 .+ theta2)) .*(sin.(theta2).*cos.(theta1) .- sin.(theta2).*sin.(theta1).*cot.(theta1 .+ theta2))

dfdtheta2 = (1 ./ sin.(theta1 .+ theta2)).*(sin.(theta1).*cos.(theta2) .- sin.(theta2).*sin.(theta1).*cot.(theta1 .+ theta2))

dgdtheta1 = (1 ./ sin.(theta1 .+ theta2)).*(-sin.(theta2).*sin.(theta1) .- sin.(theta2).*cos.(theta1).*cot.(theta1 .+ theta2))

dgdtheta2 = (1 ./ sin.(theta1 .+ theta2)).*(cos.(theta2).*cos.(theta1) .- sin.(theta2).*cos.(theta1).*cot.(theta1 .+ theta2));


In [86]:


varX = (sigma ^ 2) .* (dfdtheta1 .^ 2) .+ (sigma ^ 2) .* (dfdtheta2 .^ 2)
varY = (sigma ^ 2) .* (dgdtheta1 .^ 2) .+ (sigma ^ 2) .* (dgdtheta2 .^ 2)

covXY = (sigma ^ 2) .* (dfdtheta1 .* dgdtheta1) .+ (sigma ^ 2) .* (dfdtheta2 .* dgdtheta2);


In [87]:
varVx = similar(varX)
varVx[1] = 0
for i in 2:length(varX)
     varVx[i] = varX[i] + varX[i-1]
end

varVy = similar(varY)
varVy[1] = 0
for i in 2:length(varY)
     varVy[i] = varY[i] + varY[i-1]
end



In [88]:

xk1 = [x[2], y[2], vx[2], vy[2]]
xkp2 = A*xk1
Pk1 = [varX[2] covXY[2] 0 0; covXY[2] varY[2] 0 0; 0 0 varVx[2] 0; 0 0 0 varVy[2]]
Pkp2 = A*Pk1*A' 
Pkp2[3] = 0
Pkp2[4] = 0
Pkp2[7] = 0
Pkp2[8] = 0
Pkp2[9] = 0
Pkp2[10] = 0
Pkp2[12] = 0
Pkp2[13] = 0
Pkp2[14] = 0
Pkp2[15] = 0
R2 = [varX[3] covXY[3] 0 0; covXY[3] varY[3] 0 0; 0 0 varVx[3] 0; 0 0 0 varVy[3]]
K2 = (Pkp2' + R2)\(Pkp2)
yk2 = [x[3], y[3], vx[3], vy[3]]
xk2 = xkp2 + K2*(yk2 - xkp2)


4-element Array{Float64,1}:
  1.3735560481433202
  0.4928744930964556
  0.02976214662637547
 -0.4903110318956163

In [89]:
Pk2 = (I(4) - K2)*Pkp2*((I(4) - K2)') + K2*R2*K2'

xkp3 = A*xk2
Pkp3 = A*Pk2*A'
Pkp3[3] = 0
Pkp3[4] = 0
Pkp3[7] = 0
Pkp3[8] = 0
Pkp3[9] = 0
Pkp3[10] = 0
Pkp3[12] = 0
Pkp3[13] = 0
Pkp3[14] = 0
Pkp3[15] = 0
R3 = [varX[4] covXY[4] 0 0; covXY[4] varY[4] 0 0; 0 0 varVx[4] 0; 0 0 0 varVy[4]]
K3 = (Pkp3 + R3)\Pkp3
yk3 = [x[4], y[4], vx[4], vy[4]]
xk3 = xkp3 + K3*(yk3 - xkp3)


4-element Array{Float64,1}:
  0.5598991133805927
 -0.34979590201174793
 -0.4504428652633242
 -0.5845008919425738

In [90]:


trkal = Pk2[1] + Pk2[6]
detkal = Pk2[1] * Pk2[6] - Pk2[2]^2
lambdakal = (trkal + (trkal^2 - 4*detkal)^(0.5))/2

trdelta = R2[1] + R2[6]
detdelta = R2[1] * R2[6] - R2[2]^2
lambdadelta = (trdelta + (trdelta^2 - 4*detdelta)^(0.5))/2

Ddelta = (5.991*lambdadelta)^0.5
Ddeltatrue = ((x[3] - truex[3])^2 + (y[3] - truey[3])^2)^0.5

Dkal = (5.991*lambdakal)^0.5
Dkaltrue = ((xk2[1] - truex[3])^2 + (xk2[2] - truey[3])^2)^0.5;

In [91]:
Pk3 = (I(4) - K3)*Pkp3*((I(4) - K3)') + K3*R3*K3'
trkal = Pk3[1] + Pk3[6]
detkal = Pk3[1] * Pk3[6] - Pk3[2]^2

lambdakal = (trkal + (trkal^2 - 4*detkal)^(0.5))/2


trdelta = R3[1] + R3[6]
detdelta = R3[1] * R3[6] - R3[2]^2
lambdadelta = (trdelta + (trdelta^2 - 4*detdelta)^(0.5))/2

@show Ddelta = (5.991*lambdadelta)^0.5
@show Ddeltatrue = ((x[4] - truex[4])^2 + (y[4] - truey[4])^2)^0.5
@show Dkal = (5.991*lambdakal)^0.5
@show Dkaltrue = ((xk3[1] - truex[4])^2 + (xk3[2] - truey[4])^2)^0.5
@show Dkal < Ddelta
@show Dkaltrue < Ddeltatrue
@show Dkaltrue < Dkal

Ddelta = (5.991lambdadelta) ^ 0.5 = 0.41421176792858194
Ddeltatrue = ((x[4] - truex[4]) ^ 2 + (y[4] - truey[4]) ^ 2) ^ 0.5 = 0.33358998183360655
Dkal = (5.991lambdakal) ^ 0.5 = 0.38480116172450896
Dkaltrue = ((xk3[1] - truex[4]) ^ 2 + (xk3[2] - truey[4]) ^ 2) ^ 0.5 = 0.3123569083854453
Dkal < Ddelta = true
Dkaltrue < Ddeltatrue = true
Dkaltrue < Dkal = true


true

In [92]:
@show Aobs = [x[1], y[1]]
@show Bobs = [x[2], y[2]]
@show Cobs = [x[3], y[3]]
@show Dobs = [x[4], y[4]]
@show Ckal = xk2[1:2]
@show Dkal = xk3[1:2]

Aobs = [x[1], y[1]] = [1.2148194479767016, 1.4213916746348352]
Bobs = [x[2], y[2]] = [1.43661693418501, 1.0283612159026274]
Cobs = [x[3], y[3]] = [1.3207628820943922, 0.49430322552859923]
Dobs = [x[4], y[4]] = [0.46675024559440764, -0.16506244231053838]
Ckal = xk2[1:2] = [1.3735560481433202, 0.4928744930964556]
Dkal = xk3[1:2] = [0.5598991133805927, -0.34979590201174793]


2-element Array{Float64,1}:
  0.5598991133805927
 -0.34979590201174793

In [93]:
R1 = [varX[2] covXY[2] 0 0; covXY[2] varY[2] 0 0; 0 0 varVx[2] 0; 0 0 0 varVy[2]]

4×4 Array{Float64,2}:
 0.133801   0.0309884  0.0       0.0
 0.0309884  0.0209031  0.0       0.0
 0.0        0.0        0.276683  0.0
 0.0        0.0        0.0       0.0707034

In [94]:
@show theta1ex = copy(theta1);
@show theta2ex = copy(theta2);
@show xex = copy(x);
@show yex = copy(y);

theta1ex = copy(theta1) = [0.7071984408742489, 0.9495306736556037, 1.2126782279102353, 1.9107119064856337]
theta2ex = copy(theta2) = [1.9046853397244403, 1.5905354316673237, 1.2051330886436615, 0.38104291261978]
xex = copy(x) = [1.2148194479767016, 1.43661693418501, 1.3207628820943922, 0.46675024559440764]
yex = copy(y) = [1.4213916746348352, 1.0283612159026274, 0.49430322552859923, -0.16506244231053838]


In [98]:
R3

4×4 Array{Float64,2}:
  0.0242784   -0.00981542  0.0      0.0
 -0.00981542   0.00654028  0.0      0.0
  0.0          0.0         0.10028  0.0
  0.0          0.0         0.0      0.0174324